In [3]:
from gtts import gTTS
import os
import fitz
import numpy as np
import AO3
import PySimpleGUI as sg
sg.theme('DarkRed1')

'DarkRed1'

Настроить говорение
Настроить разные языки
Написать pdf
Написать вывод ошибок

In [6]:
def from_pdf(pdf_document):
    if '.pdf' not in pdf_document:
        print('Wrong file format!')
        return
    
    doc = fitz.open(pdf_document)  

    book_marks = np.array(doc.getToC())[1:,2]
    book_marks = np.append(book_marks, doc.pageCount)

    for i in range(len(book_marks) - 1):
        chapter_text = str()

        for j in range(int(book_marks[i]), int(book_marks[i+1])):
            page = doc.loadPage(j - 1)  
            page_text = page.getText("text")  
            chapter_text += page_text 

        i += 1

        chapter_text = chapter_text.replace('\n', ' ')

        ts = gTTS(chapter_text)
        ts.save(pdf_document.split('.pdf')[0] + ' CH' + str(i) + '.mp3')

        print('Chapter %i is saved' % i)

    print('Work Saved!')
    
    return

In [7]:
def from_url(work_id_str, dir_out):
    work_id_str = work_id_str.strip(' ')
    work_id = int(work_id_str)
    
    try:
        work = AO3.Work(work_id)
    except: 
        print('Work does not exist or work is restricted. Please check url or use pdf option.')
        return
    
    work.load_chapters()
    
    print(work.title)
    print('Number of chapters:', work.chapters)
    
    for i in range(work.chapters):
        i += 1
        text = work.get_chapter_text(i)

        tts = gTTS(text)
        tts.save(os.path.join(dir_out, work_id_str + 'CH' + str(i) + '.mp3'))

        print('Chapter %i is saved' % i)
    
    return work.title, work.chapters

In [8]:
# All the stuff inside your window.
layout = [ [sg.Text('Greetings! This is a program to convert AO3 fics to .mp3 using automatic text-to-speach:')],
            [sg.Text(' ')],
            [sg.Text('Enter fic id on AO3:')], 
            [sg.InputText(key="-URL-")],
            [sg.Text('or')], 
            [sg.Text('Choose pdf file downloaded from AO3 on your computer:')], 
            [sg.InputText(key="-PDF-"), sg.FileBrowse()],
            [sg.Text(' ')],
            [sg.Text('Where to save:')], 
            [sg.InputText(key="-DIR-"), sg.FolderBrowse()],
            [sg.Button('Generate'), sg.Button('Cancel')]]

# Create the Window
window = sg.Window('AO3 speaker v1', layout)

# Event Loop to process "events" and get the "values" of the inputs
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event == 'Cancel': # if user closes window or clicks cancel
        break
    ##################    
    output_dir = values['-DIR-']
    work_id = values['-URL-']
    pdf_adress = values['-PDF-']
        
    if work_id == '' and pdf_adress == '':
        print('Please enter fic credentials!')
        
    if output_dir == '':
        print('Please shoose where to save audio!')

    from_url(work_id, output_dir)
    from_pdf(pdf_adress)
    
    print()
window.close()